In [ ]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import cv2
import imghdr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.layers import Activation, Flatten, Dropout, Dense
# from keras.layers.core import Flatten
# from keras.layers.core import Dropout
# from keras.layers.core import Dense
from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras.utils import np_utils
from imutils import build_montages
from imutils import paths
import matplotlib.pyplot as plt
from keras.models import Sequential
# from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from keras.layers.convolutional import MaxPooling2D

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# cwd = os.getcwd()
# os.chdir(cwd)
#data_dir=r"C:\Users\HP\Desktop\Online Courses\IBM Machine Learning Engineer\4. Deep Learning and Reinforcement Learning\Week 9\projects\nemophea\origional data\chest_xray\chest_xray"
data_dir=r"C:\Users\BAYEE\Desktop\Online Courses\IBM Machine Learning Engineer\4. Deep Learning and Reinforcement Learning\Week 9\projects\nemophea\origional data\chest_xray\chest_xray"
#print(os.listdir(r"C:\Users\HP\Desktop\Online Courses\IBM Machine Learning Engineer\4. Deep Learning and Reinforcement Learning\Week 9\projects\nemophea\origional data\chest_xray"))
os.listdir(os.path.join(data_dir, 'test'))
# Any results you write to the current directory are saved as output.

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

trainNormal = []
for i in os.listdir(os.path.join(data_dir, 'train', 'NORMAL')):
    i = os.path.join(data_dir, 'train', 'NORMAL', i)
    tip = imghdr.what(i)
    if tip in image_exts: 
        trainNormal.append(i)
        
trainPneumonia = []
for i in os.listdir(os.path.join(data_dir, 'train', 'PNEUMONIA')):
    i = os.path.join(data_dir, 'train', 'PNEUMONIA', i)
    tip = imghdr.what(i)
    if tip in image_exts: 
        trainPneumonia.append(i)
        
testNormal = []
for i in os.listdir(os.path.join(data_dir, 'test', 'NORMAL')):
    i = os.path.join(data_dir, 'test', 'NORMAL', i)
    tip = imghdr.what(i)
    if tip in image_exts: 
        testNormal.append(i)
        
testPneumonia = []
for i in os.listdir(os.path.join(data_dir, 'test', 'PNEUMONIA')):
    i = os.path.join(data_dir, 'test', 'PNEUMONIA', i)
    tip = imghdr.what(i)
    if tip in image_exts: 
        testPneumonia.append(i)



In [ ]:
Pneumonia, Normal, All = [], [], []

for i, j in zip(trainPneumonia, trainNormal):
    Pneumonia.append(i)
    Normal.append(j)
    All.append(i)
    All.append(j)

for i, j in zip(testPneumonia, testNormal):
    Pneumonia.append(i)
    Normal.append(j)
    All.append(i)
    All.append(j)

In [ ]:
print("Loading images...")
# imagePaths = list(paths.list_images('dataset'))
imagePaths = All
# print(imagePaths)
print("Loaded")

In [ ]:
data = []
labels = []

for imagePath in imagePaths:
    # Extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # load the image, and resize it to be a fixed 100x100 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (100, 100))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

In [ ]:
# # Converting data into Numpy Array, scaling it to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
# # Reshaping for channel dimension
data = data.reshape((data.shape[0], data.shape[1], data.shape[2], 3))

In [ ]:
# Encoding Labels
LE = LabelEncoder()
labels = LE.fit_transform(labels)

# One-hot Encoding
labels = np_utils.to_categorical(labels, 2)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, stratify=labels, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
# model = Sequential()
# model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(255,255,3)))
# model.add(MaxPooling2D())
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Conv2D(16, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(255, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5)) # DROPOUT
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])

In [ ]:
epochs = 20
ConvNet = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=epochs, verbose=1)
print("Training Complete")

# from keras.models import load_model

# model.save('myModel.h5')
# print("Model Saved")

In [ ]:
# model = load_model('myModel.h5')
print("Evaluating the network")
predictions = model.predict(X_test, batch_size=32)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
confusion_matrix()

In [ ]:
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=LE.classes_))

In [ ]:
def predict_kneumonia(image):
    img = cv2.imread(image)
    plt.imshow(img)
    plt.show()
    resize = tf.image.resize(img, (256,256))
    yhat = model.predict(np.expand_dims(resize/255, 0))
    if yhat > 0.5: 
        print(f'Predicted Results : Kneumonia')
    else:
        print(f'Predicted Results : Normal')